In this section we are going to understand how we can built a simple chain using LangGraph that uses 4 importtant concept
- How to use chat message as graph state
- How to use chat model in graph node.
- How to bind tools to our LLM in chat model.
- How to execute the tool call in our graph node.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint # used for good representation of message

messages = [AIMessage(content=f"Hello, How can I help you",name='LLMModel')]
messages.append(HumanMessage(content=f"I want to learn coding",name='Pradum'))
messages.append(AIMessage(content=f"Which programming language you want to learn",name="LLMModel"))
messages.append(HumanMessage(content=f"I want to learn python programming language",name='Pradum'))

for message in messages:
    message.pretty_print()

### Chat Models

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")
response = llm.invoke(messages)
print(response)

In [ ]:
# to see the metadata
response.response_metadata

In [ ]:
response.pretty_print()

### Tools

In [ ]:
def add(a:int,b:int) -> int:
    """ Add a and b
    Args:
        a (int) :first int
        b (int): second int

    Returns:
        int
    
    """
    return a+b


In [ ]:
llm

In [ ]:
### Binding tool with llm
llm_with_tools = llm.bind_tools([add])

tool_call=llm_with_tools.invoke([HumanMessage(content=f"What is 2 plus 2",name="Pradum")])

In [ ]:
tool_call.tool_calls

In [ ]:
## Using message as State
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class State(TypedDict):
    message:list[AnyMessage] # AnyMessage contains all type of messages


### Reducers
- generally new message overwrite the previous one that why to overcome that particular issue we use Reducers with the help of this you can control the updation of messages
- add_message is also a type of reducers that help to add the new message with old one without override it.
- Annotated allows us to add extra metadata with type hint

In [ ]:
from langgraph.graph.message import add_messages
from typing import Annotated
class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

Reducer with add_message

In [ ]:
initial_messages = [AIMessage(content=f"Hello, How can I help you",name='LLMModel')]
initial_messages.append(HumanMessage(content=f"I want to learn coding",name='Pradum'))
initial_messages

In [ ]:
ai_message=AIMessage(content=f"Which programming language you want to learn",name="LLMModel")
ai_message

In [ ]:
## Reducer add_message is to append instead of override
add_messages(initial_messages,ai_message)

In [ ]:
# Chatbot node funtionality
def llm_tool(state:State):
    return {"messages":[llm_with_tools.invoke(state['messages'])]}

In [ ]:
from IPython.display import display, Image
from langgraph.graph import StateGraph,START,END

builder=StateGraph(State)
builder.add_node("llm_tool",llm_tool)

builder.add_edge(START,"llm_tool")
builder.add_edge("llm_tool",END)

graph=builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages=graph.invoke({"messages":"What is 2 plus 2"})

for message in messages['messages']:
    message.pretty_print()

In [ ]:
tools= [add]

In [ ]:
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

builder = StateGraph(State) 

builder.add_node("llm_tool",llm_tool)
builder.add_node("tools",ToolNode(tools))

# Add Edges
builder.add_edge(START,"llm_tool")
builder.add_conditional_edges(
    "llm_tool",
    # If the latest message(result) from assistamt is a tool call -> tools_condition routes to tools
    # If the latest message(result) from assistamt is not tool call -> tools_condition routes t o END
    tools_condition

    )
builder.add_edge("tools",END)
graph = builder.compile()


In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages=graph.invoke({"messages":"What is 2 plus 2"})

for message in messages['messages']:
    message.pretty_print()

In [ ]:
messages=graph.invoke({"messages":"What is Generative AI"})

for message in messages['messages']:
    message.pretty_print()